# **PROYECTO SCHUTZ EN PYTHON PARA CREACIÓN DE DASHBOARD 1 - PARTE 1**

# 1. IMPORTACIÓN DE LIBRERÍAS PARA EL PROYECTO

In [158]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from itertools import zip_longest
import io
import requests
from io import BytesIO
import pandas as pd
import chardet
import re
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import set_with_dataframe
from urllib.request import FancyURLopener
from urllib.request import urlopen
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import ssl
import warnings


In [159]:
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

## 2. LECTURA DE DATAFRAMES

### 2.1 LECTURA "LIBRO MAYOR"

In [160]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Libro%20Mayor%202023.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_libro_mayor = pd.DataFrame(data)

# Imprimir el dataframe
df_libro_mayor.sample(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia
2321,4101010-000,2023,2,2023-02-20T00:00:00-03:00,CIN,535,Arriendo Maquina Electrofusion,P-1712,O19501068,180000,0,49504
6193,4150002-000,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES DIRECTAS PROYECTO,P-1684,17378948,433344,0,NaN
834,4101010-000,2023,1,2023-01-03T00:00:00-03:00,CIN,55,Telefono Veo Duox Plus Fermax F03444,P-1682,C04082152,649950,0,48704
1701,4101010-000,2023,1,2023-01-31T00:00:00-03:00,CIN,1278,Cinta Aisladora 3M Temflex 1500,P-1694,I16430107,3232,0,49213
5778,4101010-000,2023,5,2023-05-23T00:00:00-04:00,CIN,1211,"Perno anclaje 3/8"" x 2 3/4""",P-1715,I15410044,24400,0,51361


In [161]:
df_libro_mayor.to_csv('archivos_respaldo/df_libro_mayor_1.csv', index = False)

### 2.2 LECTURA "DESPACHOS REALIZADOS"

In [162]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Despachos%20Realizados.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_desp_realizados = pd.DataFrame(data)

# Imprimir el dataframe
df_desp_realizados.sample(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
10500,P-Proyectos,VCO,2023-06-02T00:00:00-04:00,51493,None,H21571172,1,Guante Multiflex Negro,UN,755,755
12550,P-1699,VCO,2023-04-20T00:00:00-03:00,50623,1075,C06112033,70,Carga 75 ohm terminal F SIN BLOQUEO,UN,258,18060
4418,P-Proyectos,VCO,2022-01-07T00:00:00-03:00,44233,None,H21570768,1,Anteojo Seguridad (antiparras),UN,575,575
18378,P-1718,VCO,2023-05-17T00:00:00-04:00,51253,1093,P08200280,7,"Tapa gorro Ø2"" ranurado",UN,719,5033
14056,P-1698,VCO,2022-05-10T00:00:00-03:00,45378,1042,P08210960,8,"Manometro Seco 0-14 Bar 4"" 300 lb UL/FM",UN,6005,48040


In [163]:
df_desp_realizados.to_csv('archivos_respaldo/df_desp_realizados_1.csv', index = False)

### 2.3 LECTURA "CODIGOS DE PROYECTO"

In [164]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\codigos%20de%20proyectos.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_cod_proyecto = pd.DataFrame(data)

# Imprimir el dataframe
df_cod_proyecto.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE
234,P-1644,Mirador de San Cristobal,EBCO S.A.
136,P-1595,Hogares,Empresa Constructora Vascro SPA
173,P-1501,Marin II B,Constructora Novatec S.A
217,P-1551,Edificio Donatello,SACYR CHILE S.A.
272,P-1602,Altum Concha y Toro,CONSTRUCTORA NOVATEC EDIFICIOS S.A


In [165]:
df_cod_proyecto.to_csv('archivos_respaldo/df_cod_proyecto_1.csv', index = False)

### 2.4 LECTURA "PROYECTADOS"

In [166]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso

# Obtener el mes actual de acuerdo a la fecha actual, y el resultado se convierte en el nombre del MES para luego buscar aquel nombre en el archivo.
mes_actual = datetime.date.today().month
print(mes_actual)

if mes_actual == 1:
    mes_actual2 = "ENERO"
elif mes_actual == 2:
    mes_actual2 = "FEBRERO"
elif mes_actual == 3:
    mes_actual2 = "MARZO"
elif mes_actual == 4:
    mes_actual2 = "ABRIL"
elif mes_actual == 5:
    mes_actual2 = "MAYO"
elif mes_actual == 6:
    mes_actual2 = "JUNIO"
elif mes_actual == 7:
    mes_actual2 = "JULIO"
elif mes_actual == 8:
    mes_actual2 = "AGOSTO"
elif mes_actual == 9:
    mes_actual2 = "SEPTIEMBRE"
elif mes_actual == 10:
    mes_actual2 = "OCTUBRE"
elif mes_actual == 11:
    mes_actual2 = "NOVIEMBRE"
else:
    mes_actual2 = "DICIEMBRE"

print(mes_actual2)

6
JUNIO


In [167]:
ssl._create_default_https_context = ssl._create_unverified_context

# Lectura de presupuestos desde dropbox
df_proyectado = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_actual2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.5 LECTURA "LIBRO MAYOR AÑOS ANTERIORES"

In [168]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

In [169]:
spr = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Fx1nerTu10PZPOwknLcQuri_I6ARat5h7zuq4P3vypg/edit#gid=1097918290')
recipients = spr.sheet1.get_all_values()

# Se el string creado en un dataframe
df_libro_mayor_old = pd.DataFrame(recipients)

# se reasignan los headers en el dataframe.
new_header = df_libro_mayor_old.iloc[0] #grab the first row for the header
df_libro_mayor_old = df_libro_mayor_old[1:] #take the data less the header row
df_libro_mayor_old.columns = new_header #set the header row as the df header

# Se conservan solo las columnas de interés
#df_seguridad_v3 = df_seguridad_v3[["Nº Cédula","Nombres y apellidos","APRUEBA FASE DE CALIFICACIÓN ?","PUNTOS VIGENTES","APRUEBA FASE DE EVALUACIÓN ?","RESULTADO FINAL"]]

print("Las dimensiones del dataframe son :",df_libro_mayor_old.shape)
df_libro_mayor_old.sample(5)

Las dimensiones del dataframe son : (17368, 13)


,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
16173,4150002-000,2022,8,8/31/22 0:00,AJC,1668,REMUNERACIONES DIRECTAS PROYECTO,P-1704,15730307,0,1017572,0,
6689,4101010-001,2021,2,2/1/21 0:00,CIN,878,Conduit Flexible 20mm Metalico con PVC (,S-1601,T14370414,40551,24750,0,
11959,4101010-000,2022,7,7/18/22 0:00,CIN,773,Spray Galvanizado en Frío,P-1685,I15412495,46314,5958,0,
4609,4101010-000,2021,7,7/19/21 0:00,CIN,644,Switch 4+2 puertos Dahua PFS3006-4ET-60,P-1658,C03051971,42641,36200,0,
9451,4101010-000,2022,2,2/28/22 0:00,CIN,811,"Sirena 15 W con gabinete, baliza y tampe",P-1675,S11300567,44640,170100,0,


In [170]:
df_libro_mayor_old.to_csv('archivos_respaldo/df_libro_mayor_old.csv', mode='a')

### 2.6 LECTURA "PLANILLA RESUMEN - PRESPUESTOS"

In [171]:
# Lectura de presupuestos desde dropbox
df_presupuesto = pd.read_excel("https://www.dropbox.com/s/miow7p9wl7t3jsr/PLANILLA%20RESUMEN.xlsb.xlsx?dl=1",sheet_name= "PROYECTOS" ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_presupuesto.head(5)

,PROYECTO,OBRA,RUT,CONSTRUCTORA,FECHA INICIO,FECHA TERMINO,ULTIMO EP PRESENTADO,AVANCE EN OBRA,MONTO EN $,$ POR FACTURAR,...,217465.036046312,217465.036046312.1,217465.036046312.2,217465.036046312.3,0,0.1,0.2,0.3,0.4,0.5
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,8111745112.45,3283770104.37,...,1,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00
1,P-1632,Chorrillos Clorox,76.108.978-1,SuKsa S.A,2019-02-04 00:00:00,2019-05-31,NaN,0.99,138107706.03,915642.26,...,217465.04,217465.04,217465.04,217465.04,0.00,0.00,0.00,0.00,0.00,0.00
2,P-1638,Colon,78134800-7,Grevia,2019-08-13 00:00:00,2020-03-27,NaN,1.00,43206282.00,2177592.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P-1645,Los 3 Antonios,77.710.830-1,EDICASA,2019-08-31 00:00:00,NaT,NaN,0.98,18150000.00,208732.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P-1647,Bodega corrosivos TW,76.564.630-8,Inmobiliaria y Rentas Logisticas S.A,NaN,NaT,NaN,0.58,29482131.00,12437344.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
df_presupuesto.to_csv('archivos_respaldo/df_presupuesto.csv', mode='a')

### 2.7 LECTURA "TABLA- PRESPUESTOS"

In [173]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Presupuesto Proyectos.txt'
response = requests.get(url)
query_result = response.text
soup = BeautifulSoup(response.content, 'xml')
rows = soup.find_all('Table')


data = []
for row in rows:
    values = [cell.text for cell in row.find_all()]
    data.append(values)

df_tabla_prespuesto = pd.DataFrame(data, columns=['CODIGO_PROYECTO','PARTIDA_PRESUPUESTARIA','PRODUCTO','UNIDAD','CANTIDAD','COSTO_UNITARIO','COSTO_TOTAL'])

# Exportación a excel
df_tabla_prespuesto.to_csv('df_tabla_prespuesto.csv', index = False)

df_tabla_prespuesto.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490,428490
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866,1022434
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846,95384
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608,138224
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200,353600


## 3. - TRATAMIENTO DE DATAFRAMES

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR

#### 3.1.1 - Exploración inicial de los datos

In [174]:
df_libro_mayor.shape

(7145, 12)

In [175]:
df_libro_mayor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7145 entries, 0 to 7144
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Cta_Contable     7145 non-null   object
 1   Año              7145 non-null   object
 2   Mes              7145 non-null   object
 3   Fecha            7120 non-null   object
 4   Tipo_Comp        7145 non-null   object
 5   Num_Comp         7145 non-null   object
 6   Glosa            7145 non-null   object
 7   Primer_Analisis  7142 non-null   object
 8   Referencia       7145 non-null   object
 9   Debito           7145 non-null   object
 10  Credito          7145 non-null   object
 11  Num_Referencia   5941 non-null   object
dtypes: object(12)
memory usage: 670.0+ KB


#### 3.1.2 - Estandarización campo "Debito" y Crédito

In [176]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor['Debito_2'] = pd.to_numeric(df_libro_mayor['Debito'], errors='coerce')
df_libro_mayor['Credito_2'] = pd.to_numeric(df_libro_mayor['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor['Credito_2'] = df_libro_mayor['Credito_2'].fillna(0)

df_libro_mayor.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.3 - Creación campo Codigo_proyecto

In [177]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Primer_Analisis']

cols = df_libro_mayor.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('Codigo_proyecto'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor = df_libro_mayor.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Codigo_proyecto'].str.upper()

df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.4 - Creación de campo total

In [178]:
df_libro_mayor['TOTAL'] = df_libro_mayor['Credito_2'] - df_libro_mayor['Debito_2']
df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0,-4081983
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0,-6464848
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0,-4673685
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0,-1570055
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0,-3613652


#### 3.1.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [179]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return 'INGRESO'
    elif cta_contable == "4101010-000":
        return 'COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return 'COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return 'COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor['Tipo_cuenta'] = df_libro_mayor['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
5345,P-1695,4101010-000,2023,5,2023-05-10T00:00:00-03:00,CIN,531,Citofono GT-1D Aiphone,P-1695,C04080111,1486803,0,51059,1486803,0,-1486803,COSTO MATERIALES
4531,P-1688,4101010-000,2023,4,2023-04-21T00:00:00-03:00,CIN,1008,Cinta Permace 3/4,P-1688,I16430109,1623,0,50649,1623,0,-1623,COSTO MATERIALES
4838,P-1707,4101010-000,2023,5,2023-05-03T00:00:00-03:00,CIN,47,ANTENA TERRESTRE DAT BOSS MIX BI/BIII/UH,P-1707,C06112981,86660,0,50825,86660,0,-86660,COSTO MATERIALES
4500,P-1709,4101010-000,2023,4,2023-04-20T00:00:00-03:00,CIN,916,grabados para rack,P-1709,O19501068,21500,0,50634,21500,0,-21500,COSTO MATERIALES
2097,P-1692,4101010-000,2023,2,2023-02-13T00:00:00-03:00,CIN,316,Conector RG11 para crimpar,P-1692,C08732769,106320,0,49408,106320,0,-106320,COSTO MATERIALES


In [180]:
# EXPLORACIÓN DE DATOS

df_libro_mayor_piv = pd.pivot_table(df_libro_mayor, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_piv = pd.DataFrame(df_libro_mayor_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
2,COSTO MANO DE OBRA,2023,3,0,96802066,-96802066
18,INGRESO,2023,5,557296360,155388906,401907454
4,COSTO MATERIALES,2023,1,2565889,126171357,-123605468
5,COSTO MATERIALES,2023,2,3348963,83938903,-80589940
17,INGRESO,2023,4,408889221,61678702,347210519
23,OTRA,2023,4,3,26395039,-26395036
15,INGRESO,2023,2,231086456,0,231086456
7,COSTO MATERIALES,2023,4,3247244,151372142,-148124898
21,OTRA,2023,2,0,22347286,-22347286
14,INGRESO,2023,1,449490833,191247845,258242988


#### 3.1.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [181]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor2 = df_libro_mayor.copy()

In [182]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-PROYECTOS' 'P-1669'
 'P-1667' 'P-1668' 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658'
 'P-1660' 'P-1644' 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653'
 'P-1673' 'P-1683' 'P-1661' 'P-1612' 'P-1728' 'P-SERVICIOS' 'P-1639'
 'P-1674' 'P1718' 'P-1620' 'P-1603' 'P-1626' 'P-1649' 'P-1685' 'P-1632'
 'P-1643' 'P-1666' 'P-1656' '15314868-6' 'P-1717' 'P-1663' '78540190-5'
 '5555555-9' 'P-OFICINA' 'P-ANALISIS PROYE' 'P-1557' 'P-1591' '18800250-1'
 '13056802-5' '16411600-K' nan]
Dimensiones del dataframe antes de la reducción : 
(7

##### 3.1.6.1 - Se Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [183]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor2_2 = df_libro_mayor2.copy()

# Filtrar el DataFrame
df_libro_mayor2_2 = df_libro_mayor2_2[(df_libro_mayor2_2['Codigo_proyecto'] == 'P-OFICINA') & (df_libro_mayor2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor2_2

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
6852,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN
6853,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
6854,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN
6855,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
6856,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292504,0,NaN,2292504,0,-2292504,COSTO SUPERVISIÓN
6857,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
6858,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN
6859,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN


In [184]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor2 = df_libro_mayor2.dropna(subset=['Codigo_proyecto'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask1 = df_libro_mayor2['Codigo_proyecto'].isin(valores_a_eliminar)

# Crea una máscara booleana que identifica las filas que comienzan con un número
mask2 = df_libro_mayor2['Codigo_proyecto'].str[0].str.isnumeric()

# Une las dos máscaras con el operador OR
mask = mask1 | mask2

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor2 = df_libro_mayor2[~mask]


# Verifica si el valor 'P1718' o valore similares están presentes en la columna Codigo_proyecto
if 'P1718' in df_libro_mayor2['Codigo_proyecto'].values:
    # Reemplaza el valor 'P1718' por 'P-1718' en la columna Codigo_proyecto
    df_libro_mayor2['Codigo_proyecto'] = df_libro_mayor2['Codigo_proyecto'].replace('P1718', 'P-1718')


In [185]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-1669' 'P-1667' 'P-1668'
 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658' 'P-1660' 'P-1644'
 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653' 'P-1673' 'P-1683'
 'P-1661' 'P-1612' 'P-1728' 'P-1639' 'P-1674' 'P-1620' 'P-1603' 'P-1626'
 'P-1649' 'P-1685' 'P-1632' 'P-1643' 'P-1666' 'P-1656' 'P-1717' 'P-1663'
 'P-1557' 'P-1591']
Dimensiones del dataframe antes de la reducción : 
(6773, 17)


#### 3.1.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [186]:
df_libro_mayor2['Fecha2'] = pd.to_datetime(df_libro_mayor2['Año'].astype(str) + '-' + df_libro_mayor2['Mes'].astype(str) + '-1')
df_libro_mayor2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
5730,P-1723,4101010-000,2023,5,2023-05-19T00:00:00-04:00,CIN,1022,Conector Conico F RG6 para crimpar,P-1723,S09242300,99300,0,51309,99300,0,-99300,COSTO MATERIALES,2023-05-01
6323,P-1694,4150002-000,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES DIRECTAS PROYECTO,P-1694,12764949,1047291,0,NaN,1047291,0,-1047291,COSTO MANO DE OBRA,2023-02-01
3103,P-1705,4101010-000,2023,3,2023-03-16T00:00:00-03:00,CIN,604,Conector Conico F RG6 para crimpar,P-1705,S09242300,146280,0,49890,146280,0,-146280,COSTO MATERIALES,2023-03-01
3989,P-1680,4101010-000,2023,4,2023-04-13T00:00:00-03:00,CIN,447,DERIVADOR 5-2400 MHZ RIT INALAMBRICA 6D,P-1680,C08713188,9926,0,50389,9926,0,-9926,COSTO MATERIALES,2023-04-01
2427,P-1704,4101010-000,2023,2,2023-02-23T00:00:00-03:00,CIN,631,Cuchillo Cortador de tubo hasta 28 mm,P-1704,H21580573,12000,0,49550,12000,0,-12000,COSTO MATERIALES,2023-02-01


##### 3.1.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [187]:
df_libro_mayor2_2['Fecha2'] = pd.to_datetime(df_libro_mayor2_2['Año'].astype(str) + '-' + df_libro_mayor2_2['Mes'].astype(str) + '-1')
df_libro_mayor2_2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
6855,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-02-01
6854,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN,2023-02-01
6858,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN,2023-04-01
6857,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-03-01
6852,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN,2023-01-01


#### 3.1.8 - Se realiza pivote final del dataframe

In [188]:
# Se exploran las columnas:
df_libro_mayor2.columns

Index(['Codigo_proyecto', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Debito',
       'Credito', 'Num_Referencia', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object')

In [189]:
# crear pivot table sin margenes

df_libro_mayor2_piv = pd.pivot_table(df_libro_mayor2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_piv = pd.DataFrame(df_libro_mayor2_piv.to_records())

df_libro_mayor2_piv.sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
133,P-1687,INGRESO,2023-01-01,2023,1,6123707,0,6123707
103,P-1684,COSTO MANO DE OBRA,2023-01-01,2023,1,0,1071862,-1071862
140,P-1688,COSTO MANO DE OBRA,2023-04-01,2023,4,0,2009744,-2009744
328,P-1707,COSTO MATERIALES,2023-02-01,2023,2,0,803767,-803767
404,P-1716,COSTO MANO DE OBRA,2023-03-01,2023,3,0,5896052,-5896052


In [190]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor2_piv.shape

(496, 8)

##### 3.1.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [191]:
# crear pivot table sin margenes

df_libro_mayor2_2_piv = pd.pivot_table(df_libro_mayor2_2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_2_piv = pd.DataFrame(df_libro_mayor2_2_piv.to_records())

df_libro_mayor2_2_piv = df_libro_mayor2_2_piv[df_libro_mayor2_2_piv['Codigo_proyecto'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor2_2_piv['Codigo_proyecto'] = df_libro_mayor2_2_piv['Codigo_proyecto'].replace('P-OFICINA', 'TOTAL')


df_libro_mayor2_2_piv

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO SUPERVISIÓN,2023-01-01,2023,1,0,4336206,-4336206
1,TOTAL,COSTO SUPERVISIÓN,2023-02-01,2023,2,0,4016206,-4016206
2,TOTAL,COSTO SUPERVISIÓN,2023-03-01,2023,3,0,4474872,-4474872
3,TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023,4,0,5096121,-5096121


#### 3.1.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta

In [192]:
df_libro_mayor_piv2 = pd.pivot_table(df_libro_mayor2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_piv2 = pd.DataFrame(df_libro_mayor_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_piv2.insert(loc=0, column='Codigo_proyecto', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != '']

df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO MANO DE OBRA,2023-01-01,2023,1,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01,2023,2,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01,2023,3,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01,2023,4,0,111885468,-111885468
4,TOTAL,COSTO MATERIALES,2023-01-01,2023,1,2565889,125972929,-123407040
5,TOTAL,COSTO MATERIALES,2023-02-01,2023,2,3348963,83658197,-80309234
6,TOTAL,COSTO MATERIALES,2023-03-01,2023,3,2764412,88079320,-85314908
7,TOTAL,COSTO MATERIALES,2023-04-01,2023,4,3247244,150098697,-146851453
8,TOTAL,COSTO MATERIALES,2023-05-01,2023,5,8533707,176803071,-168269364
9,TOTAL,COSTO MATERIALES,2023-06-01,2023,6,2494608,34427715,-31933107


##### 3.1.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [193]:
df_libro_mayor_piv2 = pd.concat([df_libro_mayor_piv2, df_libro_mayor2_2_piv], ignore_index=True)
df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO MANO DE OBRA,2023-01-01,2023,1,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01,2023,2,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01,2023,3,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01,2023,4,0,111885468,-111885468
4,TOTAL,COSTO MATERIALES,2023-01-01,2023,1,2565889,125972929,-123407040
5,TOTAL,COSTO MATERIALES,2023-02-01,2023,2,3348963,83658197,-80309234
6,TOTAL,COSTO MATERIALES,2023-03-01,2023,3,2764412,88079320,-85314908
7,TOTAL,COSTO MATERIALES,2023-04-01,2023,4,3247244,150098697,-146851453
8,TOTAL,COSTO MATERIALES,2023-05-01,2023,5,8533707,176803071,-168269364
9,TOTAL,COSTO MATERIALES,2023-06-01,2023,6,2494608,34427715,-31933107


#### 3.1.10 - Se concatenan los 2 dataframes resultantes

In [194]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor2_piv.shape)

Dimensiones del dataframe antes de la concatenación : 
(496, 8)


In [195]:
df_libro_mayor3 = pd.concat([df_libro_mayor2_piv, df_libro_mayor_piv2])
df_libro_mayor3.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
248,P-1699,COSTO MATERIALES,2023-03-01,2023,3,0,3475458,-3475458
109,P-1684,COSTO MATERIALES,2023-03-01,2023,3,3200,717239,-714039
251,P-1699,COSTO MATERIALES,2023-06-01,2023,6,0,4987288,-4987288
96,P-1682,COSTO MATERIALES,2023-03-01,2023,3,0,459123,-459123
261,P-1700,COSTO MATERIALES,2023-01-01,2023,1,198467,2257293,-2058826


In [196]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe después de la concatenación : 
(516, 8)


#### 3.1.11 - Se crean las filas que permiten tener siempre todas las opciones de Tipo_Cuenta por cada Codigo_proyecto

In [197]:
combinaciones = []
for proyecto in df_libro_mayor3['Codigo_proyecto'].unique():
    for fecha in df_libro_mayor3['Fecha2'].unique():
        mes = pd.to_datetime(fecha).month
        año = pd.to_datetime(fecha).year
        for cuenta in ['INGRESO', 'COSTO MATERIALES', 'COSTO MANO DE OBRA', 'COSTO SUPERVISIÓN']:
            combinaciones.append((proyecto, fecha, mes, año, cuenta))

# crear un nuevo DataFrame con las combinaciones obtenidas
combinaciones_df = pd.DataFrame(combinaciones, columns=['Codigo_proyecto', 'Fecha2', 'Mes', 'Año', 'Tipo_cuenta'])

# agregar las demás columnas con valores nulos
combinaciones_df['Credito_2'] = 0
combinaciones_df['Debito_2'] = 0
combinaciones_df['TOTAL'] = 0

# Se eliminan filas con fecha en blanco.
combinaciones_df = combinaciones_df.dropna(subset=['Mes'])

combinaciones_df.sample(10)

,Codigo_proyecto,Fecha2,Mes,Año,Tipo_cuenta,Credito_2,Debito_2,TOTAL
1822,P-1712,2023-04-01,4.00,2023.00,COSTO MANO DE OBRA,0,0,0
1792,P-1709,2023-04-01,4.00,2023.00,INGRESO,0,0,0
719,P-1667,2023-05-01,5.00,2023.00,COSTO SUPERVISIÓN,0,0,0
929,P-1676,2023-01-01,1.00,2023.00,COSTO MATERIALES,0,0,0
2216,P-1726,2023-01-01,1.00,2023.00,INGRESO,0,0,0
842,P-1673,2023-04-01,4.00,2023.00,COSTO MANO DE OBRA,0,0,0
661,P-1664,2023-05-01,5.00,2023.00,COSTO MATERIALES,0,0,0
428,P-1648,2023-02-01,2.00,2023.00,INGRESO,0,0,0
822,P-1672,2023-02-01,2.00,2023.00,COSTO MANO DE OBRA,0,0,0
1434,P-1695,2023-01-01,1.00,2023.00,COSTO MANO DE OBRA,0,0,0


#### 3.1.12 - Se concatenan los 2 dataframes resultantes

In [198]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe antes de la concatenación : 
(516, 8)


In [199]:
df_libro_mayor4 = pd.concat([df_libro_mayor3, combinaciones_df])
df_libro_mayor4.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
329,P-1642,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0
35,P-1591,COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0
649,P-1664,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0
120,P-1611,INGRESO,2023-02-01,2023.00,2.00,0,0,0
884,P-1674,INGRESO,2023-05-01,2023.00,5.00,0,0,0


In [200]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor4.shape)

Dimensiones del dataframe después de la concatenación : 
(2508, 8)


#### 3.1.13 - Estandarización de campos 

In [201]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor4 = df_libro_mayor4.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor4['AÑO'] = pd.to_numeric(df_libro_mayor4['AÑO'], downcast='integer')
df_libro_mayor4['MES'] = pd.to_numeric(df_libro_mayor4['MES'], downcast='integer')

df_libro_mayor4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
506,P-1656,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0
497,P-1653,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0
764,P-1669,INGRESO,2023-02-01,2023.00,2.00,0,0,0
1420,P-1694,INGRESO,2023-06-01,2023.00,6.00,0,0,0
333,P-1707,INGRESO,2023-05-01,2023.00,5.00,7084469,0,7084469
1278,P-1689,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0
75,P-1592,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0
2059,P-1720,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0
1056,P-1681,INGRESO,2023-06-01,2023.00,6.00,0,0,0
1122,P-1684,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0


In [202]:
#df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
495,TOTAL,,NaT,NaN,NaN,2185324390,1631031003,554293387
0,TOTAL,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468
4,TOTAL,COSTO MATERIALES,2023-01-01,2023.00,1.00,2565889,125972929,-123407040
5,TOTAL,COSTO MATERIALES,2023-02-01,2023.00,2.00,3348963,83658197,-80309234
6,TOTAL,COSTO MATERIALES,2023-03-01,2023.00,3.00,2764412,88079320,-85314908
7,TOTAL,COSTO MATERIALES,2023-04-01,2023.00,4.00,3247244,150098697,-146851453
8,TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,8533707,176803071,-168269364


#### 3.1.14 Creación de columna MARGEN

In [203]:
df_libro_mayor4_2 = df_libro_mayor4.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_libro_mayor4_2.groupby(['CODIGO_PROYECTO','MES']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

    
df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(5)

,CODIGO_PROYECTO,MES,MARGEN
456,P-1723,1.00,0.00
285,P-1691,4.00,0.00
152,P-1667,3.00,0.00
242,P-1684,3.00,0.00
473,P-1725,6.00,0.00


In [204]:
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  : ", df_libro_mayor4.shape)
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          : ", df_margen.shape)
print(df_margen.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  :  (2508, 8)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          :  (498, 3)
(498, 3)


In [205]:
# CONCATENACIÓN DE DATAFRAMES

df_libro_mayor5 = pd.merge(df_libro_mayor4, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor5['CODIGO_PROYECTO'] = df_libro_mayor5['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  : ", df_libro_mayor5.shape)
print(df_libro_mayor5.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  :  (2508, 9)
(2508, 9)


In [206]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
495,1. TOTAL,,NaT,NaN,NaN,2185324390,1631031003,554293387,NaN
496,1. TOTAL,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,72482994,-72482994,0.00
497,1. TOTAL,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,82645544,-82645544,0.00
498,1. TOTAL,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,96760244,-96760244,0.00
499,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,0.00
500,1. TOTAL,COSTO MATERIALES,2023-01-01,2023.00,1.00,2565889,125972929,-123407040,0.00
501,1. TOTAL,COSTO MATERIALES,2023-02-01,2023.00,2.00,3348963,83658197,-80309234,0.00
502,1. TOTAL,COSTO MATERIALES,2023-03-01,2023.00,3.00,2764412,88079320,-85314908,0.00
503,1. TOTAL,COSTO MATERIALES,2023-04-01,2023.00,4.00,3247244,150098697,-146851453,0.00
504,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,8533707,176803071,-168269364,0.00


In [207]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
290,P-1704,COSTO MANO DE OBRA,2023-01-01,2023.00,1.00,0,7375493,-7375493,0.00
291,P-1704,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,10751091,-10751091,0.00
292,P-1704,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,12332472,-12332472,0.00
293,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,14242535,-14242535,0.00
294,P-1704,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,20073319,-20073319,0.00
295,P-1704,COSTO MATERIALES,2023-02-01,2023.00,2.00,139976,3388760,-3248784,0.00
296,P-1704,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,14768626,-14768626,0.00
297,P-1704,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,20031042,-20031042,0.00
298,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,13967551,-13967551,0.00
299,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,3973960,-3973960,0.00


In [208]:
df_libro_mayor5["CODIGO_PROYECTO"].unique()

array(['P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611', 'P-1612',
       'P-1620', 'P-1626', 'P-1632', 'P-1639', 'P-1641', 'P-1642',
       'P-1643', 'P-1644', 'P-1645', 'P-1648', 'P-1649', 'P-1653',
       'P-1656', 'P-1658', 'P-1660', 'P-1661', 'P-1663', 'P-1664',
       'P-1666', 'P-1667', 'P-1668', 'P-1669', 'P-1670', 'P-1672',
       'P-1673', 'P-1674', 'P-1675', 'P-1676', 'P-1677', 'P-1679',
       'P-1680', 'P-1681', 'P-1682', 'P-1683', 'P-1684', 'P-1685',
       'P-1686', 'P-1687', 'P-1688', 'P-1689', 'P-1690', 'P-1691',
       'P-1692', 'P-1693', 'P-1694', 'P-1695', 'P-1696', 'P-1697',
       'P-1698', 'P-1699', 'P-1700', 'P-1701', 'P-1703', 'P-1704',
       'P-1705', 'P-1706', 'P-1707', 'P-1708', 'P-1709', 'P-1712',
       'P-1713', 'P-1714', 'P-1715', 'P-1716', 'P-1717', 'P-1718',
       'P-1719', 'P-1720', 'P-1721', 'P-1722', 'P-1723', 'P-1724',
       'P-1725', 'P-1726', 'P-1727', 'P-1728', '1. TOTAL'], dtype=object)

In [209]:
df_libro_mayor5.to_csv('archivos_respaldo/df_libro_mayor5.csv', index = False)

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR SUBSETS

#### 3.1.16 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO"**

In [210]:
fecha_actual = datetime.date.today()
fecha_actual

datetime.date(2023, 6, 5)

In [211]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd = df_libro_mayor5.copy()
df_libro_mayor_ytd.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
538,P-1557,COSTO MANO DE OBRA,2023-06-01,2023.00,6.00,0,0,0,0.00
756,P-1641,INGRESO,2023-04-01,2023.00,4.00,0,0,0,0.00
1727,P-1694,COSTO SUPERVISIÓN,2023-02-01,2023.00,2.00,0,0,0,0.00
1999,P-1706,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00
662,P-1620,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00


In [212]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso

# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Filtrar los registros del dataframe que corresponden al mes actual y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month < mes_actual]

df_libro_mayor_ytd_closed.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
208,P-1695,INGRESO,2023-01-01,2023.00,1.00,2591314,0,2591314,0.00
2118,P-1713,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00
159,P-1690,INGRESO,2023-01-01,2023.00,1.00,11659330,0,11659330,0.00
839,P-1644,COSTO SUPERVISIÓN,2023-02-01,2023.00,2.00,0,0,0,0.00
1527,P-1686,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
593,P-1603,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0,0.00
1268,P-1674,INGRESO,2023-02-01,2023.00,2.00,0,0,0,0.00
112,P-1684,INGRESO,2023-03-01,2023.00,3.00,2395792,0,2395792,0.00
121,P-1686,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,4886851,-4886851,0.00
1612,P-1689,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00


In [213]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_piv = pd.pivot_table(df_libro_mayor_ytd_closed, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)
#df_libro_mayor4_piv = pd.pivot_table(df_libro_mayor4, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta'], columns = ['Fecha2'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')

df_libro_mayor_ytd_closed_piv = pd.DataFrame(df_libro_mayor_ytd_closed_piv.to_records())

df_libro_mayor_ytd_closed_piv.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
299,P-1720,INGRESO,0
27,P-1612,INGRESO,0
227,P-1699,INGRESO,94757672
22,P-1611,COSTO SUPERVISIÓN,0
10,P-1591,COSTO SUPERVISIÓN,0
29,P-1620,COSTO MATERIALES,-16400
104,P-1667,COSTO MANO DE OBRA,-5776759
145,P-1679,COSTO MATERIALES,-96622
300,P-1721,COSTO MANO DE OBRA,-7120681
194,P-1691,COSTO SUPERVISIÓN,0


#### 3.1.16-1 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO 2"**
Este dataframe es para considerar como mes cerrado 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [214]:
# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Calcular el mes que es dos meses antes del actual
mes_filtrado = mes_actual - 2 if mes_actual > 2 else (12 + mes_actual - 2) 

# Filtrar los registros del dataframe que corresponden al mes filtrado y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed_2M = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month <= mes_filtrado]

df_libro_mayor_ytd_closed_2M.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
2424,P-1726,INGRESO,2023-03-01,2023.00,3.00,0,0,0,0.00
526,P-1557,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
1702,P-1693,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
2376,P-1724,INGRESO,2023-03-01,2023.00,3.00,0,0,0,0.00
1610,P-1689,COSTO MANO DE OBRA,2023-03-01,2023.00,3.00,0,0,0,0.00
2366,P-1724,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
1838,P-1699,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
2297,P-1721,COSTO MATERIALES,2023-01-01,2023.00,1.00,0,0,0,0.00
638,P-1612,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
1670,P-1692,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00


In [215]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_2M = pd.pivot_table(df_libro_mayor_ytd_closed_2M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd_closed_2M = pd.DataFrame(df_libro_mayor_ytd_closed_2M.to_records())

df_libro_mayor_ytd_closed_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
57,P-1644,COSTO MATERIALES,204032
228,P-1700,COSTO MANO DE OBRA,-4264241
218,P-1697,COSTO SUPERVISIÓN,0
295,P-1719,INGRESO,131348
64,P-1648,COSTO MANO DE OBRA,0


#### 3.1.17 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO**

In [216]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.
 
df_libro_mayor5_mec = df_libro_mayor5.copy()
df_libro_mayor5_mec.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  : ", df_libro_mayor5_mec.shape)
df_libro_mayor5_mec.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  :  (2508, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
263,P-1700,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,5162689,-5162689,0.00
1088,P-1664,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00
1271,P-1674,COSTO SUPERVISIÓN,2023-02-01,2023.00,2.00,0,0,0,0.00


In [217]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1,P-1591,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,2483998,-2483998,0.00,LIBRO MAYOR
961,P-1656,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00,LIBRO MAYOR
664,P-1620,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR
675,P-1620,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,0.00,LIBRO MAYOR
1566,P-1687,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


In [218]:
# Se seleccionan los meses que son utiles para crear la vista de mes en curso.
# Para el real se necesita: INGRESO, COSTO MATERIALES, Y SUPERVISIÓN (ÉSTE ÚLTIMO DEL MES ANTERIOR)
# Para el proyectado se necesita: SUPERVISIÓN (DEL MES ANTERIOR)

In [219]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta


df_libro_mayor5_mec['FECHA'] = pd.to_datetime(df_libro_mayor5_mec['FECHA'])

filtro_ingreso_materiales = ((df_libro_mayor5_mec['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec['FECHA'].dt.month == datetime.now().month)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_supervision_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_mo_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec2 = df_libro_mayor5_mec[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  : ", df_libro_mayor5_mec2.shape)

df_libro_mayor5_mec2.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  :  (355, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1640,P-1690,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
558,P-1591,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
251,P-1699,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,4987288,-4987288,0.00,LIBRO MAYOR
2167,P-1715,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2024,P-1707,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2190,P-1716,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1976,P-1705,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1520,P-1685,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2048,P-1708,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1064,P-1663,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [220]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
299,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,3973960,-3973960,0.00,LIBRO MAYOR
1950,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1951,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1952,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1953,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [221]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
505,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,2494608,34427715,-31933107,0.00,LIBRO MAYOR
511,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,33681393,0,33681393,0.00,LIBRO MAYOR
2502,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2503,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2504,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2505,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.17-1 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO 2**
Este dataframe es para considerar como mes en curso de 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [222]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.

df_libro_mayor5_mec_2M = df_libro_mayor5.copy()
df_libro_mayor5_mec_2M.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (2508, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
126,P-1686,INGRESO,2023-05-01,2023.00,5.00,20143695,8335755,11807940,0.00
2031,P-1708,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
1681,P-1692,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00


In [223]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta

df_libro_mayor5_mec_2M['FECHA'] = pd.to_datetime(df_libro_mayor5_mec_2M['FECHA'])

# Filtro para INGRESO Y COSTO DE MATERIALES
mes_anterior = (datetime.now() - timedelta(days=30)).month

filtro_ingreso_materiales = ((df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_supervision_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_mo_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec_2M = df_libro_mayor5_mec_2M[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)

df_libro_mayor5_mec_2M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (446, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
179,P-1692,INGRESO,2023-05-01,2023.00,5.00,11142397,11000000,142397,0.00
470,P-1725,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,6708545,-6708545,0.00
1286,P-1675,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
749,P-1639,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00
1,P-1591,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,2483998,-2483998,0.00


In [224]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
293,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,14242535,-14242535,0.00
298,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,13967551,-13967551,0.00
304,P-1704,INGRESO,2023-05-01,2023.00,5.00,132309997,53250000,79059997,0.00
1934,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
1935,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
1948,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
1949,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00


In [225]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
499,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,0.00
504,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,8533707,176803071,-168269364,0.00
510,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,557296360,155388906,401907454,0.00
515,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,5096121,-5096121,0.00
2486,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00
2487,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
2500,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
2501,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00


#### 3.1.18 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual)**

In [226]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd2 = df_libro_mayor5.copy()
df_libro_mayor_ytd2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
637,P-1612,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
1888,P-1701,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00
1489,P-1684,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00
1599,P-1689,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00
720,P-1632,INGRESO,2023-03-01,2023.00,3.00,0,0,0,0.00
2365,P-1724,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
474,P-1725,INGRESO,2023-05-01,2023.00,5.00,5879492,0,5879492,0.00
54,P-1672,COSTO MATERIALES,2023-04-01,2023.00,4.00,303910,0,303910,0.00
844,P-1644,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
511,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,33681393,0,33681393,0.00


In [227]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv = pd.pivot_table(df_libro_mayor5, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv = pd.DataFrame(df_libro_mayor_ytd2_closed_piv.to_records())

df_libro_mayor_ytd2_closed_piv.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
330,P-1728,COSTO MATERIALES,-1822779
96,P-1663,INGRESO,0
199,P-1692,COSTO SUPERVISIÓN,0
266,P-1712,COSTO MATERIALES,-18731684
187,P-1689,COSTO SUPERVISIÓN,0
51,P-1642,COSTO SUPERVISIÓN,0
280,P-1715,INGRESO,55195577
279,P-1715,COSTO SUPERVISIÓN,0
203,P-1693,COSTO SUPERVISIÓN,0
182,P-1688,COSTO MATERIALES,-14702007


#### 3.1.18-2 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual) 2**